# Building Effective Agents (with Pydantic AI)

Examples for the agentic workflows discussed in
[Building Effective Agents](https://www.anthropic.com/research/building-effective-agents)
by [Erik Schluntz](https://github.com/eschluntz) and [Barry Zhang](https://github.com/ItsBarryZ)
of Anthropic, inspired, ported and adapted from the
[code samples](https://github.com/anthropics/anthropic-cookbook/tree/main/patterns/agents)
by the authors using [Pydantic AI](https://ai.pydantic.dev/).

## Orchestrator - Workers
Examples are based from [Intellectronica - Building Effective Agents with Pydantic AI](https://github.com/intellectronica/building-effective-agents-with-pydantic-ai)

In [1]:
%pip install -r requirements.txt
from IPython.display import clear_output ; clear_output()

In [2]:
from util import initialize, show
AI_MODEL = initialize()

import asyncio
import json
from typing import List, Dict

from pydantic import BaseModel, Field
from pydantic_ai import Agent

Available AI models:
['azure:gpt-4o', 'azure:gpt-4o-mini']

Using AI model: azure:gpt-4o
Configuring Azure AI Foundry model: gpt-4o at https://agent-workshop-yrkd.cognitiveservices.azure.com/


### Workflow: Orchestrator - Workers

An orchestrator is an agent that determines what work needs to be done
and divides it so that it can be delegated to multiple workers. Once
the work plan is ready, we run each task by a separate worker in parallel
and eventually collect the results from all workers.

> <img src="https://ai.pydantic.dev/img/pydantic-ai-dark.svg" style="height: 1em;" />
> The schemas defined by Pydantic are translated into JSON schema, which is passed on
> to the model API for defining how the result should be structured. Each API handles
> this a bit differentls, but the details are abstracted by the library. In general,
> some complexity and nesting of types is usually OK. In this case, we define a `Task`
> type, and return a list of tasks as part of `OrchestratorResponse`.
>
> Agents in Pydantic AI have some fixed definitions, like their `system_prompt`, and
> can be used to execute multiple runs, each with its own prompt. Here we define a
> worker agent instance, which can execute each task definition.

In [3]:
class Task(BaseModel):
    type: str = Field(..., description=(
        'The type of minion task approach. '
        'For example: "enthusiastic", "technical", "banana-focused", "gadget-oriented", ...'))
    description: str = Field(
        ...,
        description='Clear description for executing this minion task.'
    )


class OrchestratorResponse(BaseModel):
    analysis: str = Field(..., description=(
        'Explain your understanding of the minion task and which variations '
        'would be valuable for Gru\'s operations. Focus on how each approach serves '
        'different aspects of minion management or evil schemes.'
    ))
    tasks: List[Task] = Field(..., description="List of minion tasks")


async def orchestrate(task: str) -> Dict:
    """Process minion task by breaking it down and running subtasks in parallel."""

    orchestrator_agent = Agent(
        AI_MODEL,
        system_prompt=(
            'You are Gru\'s master orchestrator for minion operations. ',
            'Analyze this minion-related task and break it down into ',
            '2-3 distinct approaches that different types of minions could handle.',
        ),
        output_type=OrchestratorResponse,
    )
    orchestrator_response = await orchestrator_agent.run(task)
    
    analysis = orchestrator_response.output.analysis
    tasks = orchestrator_response.output.tasks
    
    show('', title='Minion Orchestrator Output')
    show(analysis, title='Analysis')
    show([task.model_dump() for task in tasks], title='Minion Tasks')
    
    # Process all the minion tasks in parallel and collect results
    worker_agent = Agent(
        AI_MODEL,
        system_prompt='You are a specialized minion worker. Generate content based on the minion task specification. Use minion language and enthusiasm where appropriate.',
    )
    worker_responses = await asyncio.gather(*[
        worker_agent.run(json.dumps(
            {'original_minion_task': task} | task_info.model_dump()
        ))
        for task_info in tasks
    ])

    for task, response in zip(tasks, worker_responses):
        show(response.output, title=f"Minion Worker Result ({task.type})")

In [4]:
await orchestrate(
    'Create a comprehensive training manual for new minions joining Gru\'s evil operations, covering banana breaks, gadget safety, and proper "BELLO!" etiquette.'
)

### Minion Orchestrator Output

### Analysis

```
Creating a comprehensive training manual for new minions involves addressing different aspects of minion life and operations. Each minion is unique, and their training should cater to their strengths and preferences to maximize efficiency and satisfaction. Three distinct approaches can be utilized to cover banana-related enjoyment, gadget-oriented vigilance, and communication skills in popular minion contexts like greetings.
```

### Minion Tasks

```
[{'description': 'Focuses on engaging minions with creative and interactive '
                 'methods. This approach includes fun exercises to practice '
                 "'BELLO!' greetings with flair, role-playing activities to "
                 'encourage banana-sharing during break times, and dynamic '
                 'demonstrations on gadget safety through playful, interactive '
                 'sessions.',
  'type': 'enthusiastic'},
 {'description': 'Designed for minions who thrive in structured environments. '
                 'This approach provides detailed guidelines and protocols on '
                 'banana nutrition, step-by-step instructions on gadget '
                 'maintenance and safety checks, and communication drills to '
                 "perfect formality and precision in 'BELLO!' greetings.",
  'type': 'technical'},
 {'description': 'Tailored for minions with a particular passion for bananas. '
                 'This strategy emphasizes the importance of banana breaks for '
                 'wellness, includes safety instructions on using gadgets '
                 'safely while distracted by banana temptations, and '
                 "encourages the integration of bananas into 'BELLO!' "
                 'etiquette to enhance enjoyment and camaraderie.',
  'type': 'banana-focused'}]
```

...

**WELCOME, NEW MINIONS, TO GRU'S BIG, BAD OPERATIONS! HEEHEE!**

*BEE-DONK-A-LEE! Time for fun, time for training, waahooo!*

---

**Chapter 1: BELLO! WITH A BANG!**

**Lesson 1: BELLO Basics!**
- Always start with a smile, like this: 😄
- Nice and loud, like you've got a megaphone: "BELLOOOOOO!"
- Wave your little hands like you're shaking them bananas at the Minion Fiesta!

**Exercise: BELLO Relay**
- Pair up with another minion. One goes “BELLO!” and the other responds, jumping up and down on one foot while smiling. Switch after each successful exchange!

**Lesson 2: BELLO Flair & Style!**
- Add a twirl or a backward somersault if feeling fancy!
- Try using different voices: Squeaky minion, gruff minion, robot minion!

---

**Chapter 2: BANANA BREAKS BONANZA!**

**Lesson 1: Bananas, Yummy Yummy!**
- Always peel your banana from the bottom, just like monkeys do. Funny, huh?
- Share a banana with a friend. Friendship is the best fruit!

**Role-Playing Activity: Banana Sharing**
- Form groups of three. One minion holds a banana, others sing “Banana! Banana!” until the first peels and shares.
- Practice different scenarios like “Hungry Minion in the Jungle” or “Banana Emergency on the Moon!”

**Lesson 2: Time to Relax & Refuel**
- Each break should be fun-filled and tasty! Grab your yellow snack, sprawl on a hammock, and tell minion jokes!

---

**Chapter 3: GADGET SAFETY FUN!**

**Lesson 1: What’s This Do?**
- Look at new gadgets with curiosity but never press the red button alone!
- Always partner up! Safety partners are the best explorers!

**Interactive Session: Gadget Game Show!**
- Pair up! One minion shows a gadget and the other guesses its use. Whether correct or hilariously wrong, high-five after each guess!
- Demonstrate correct use with over-the-top facial expressions and silly voices!

**Lesson 2: Safety First, Chaos Second!**
- Always wear your goggles. Eyes are important for seeing, especially bananas!
- Assume every button can launch a rocket! Stand back, be ready to run, and laugh!

---

**MINION SAFETY PLEDGE**

"I promise to always share my bananas, use gadgets safely, and 'BELLO!' with joy and zest. Minion friends are the best!"

*POK-KA! Heehee, time to dance!* 

Remember to have fun, stay loyal, and enjoy every mischievous minute! You’re banana-tastic! Let's go help Gru conquer the world, one giggle at a time! HEEHEE!

### Minion Worker Result (technical)

**Minion Training Manual: Welcome to Gru's Evil Operations!**

**Bello! New Minion Friends!** Welcome to the team! Here's your step-by-step guide to becoming the bestest minion in Gru's evil empire. Get ready, grab your yellow overalls and goggles, and let's get started!

---

### Section 1: Banana Break Protocol

**1.1 Banana Nutrition Essentials:**
- **Banana Benefits:** Super-duper source of energy! Keeps minions strong for mischievous tasks.
- **Consumption Guidelines:** 
  - Minimum 3 bananas daily for optimal minion performance.
  - Extra bananas granted during high-energy activities (post-mission and dance parties).

**1.2 Break Time Guidelines:**
- **Scheduled Breaks:** Observe break schedule posted in break rooms. Always prioritize safety and timing!
- **Respect Fellow Minions:** Share the banana stash and leave some for others. No banana hoarding!

**1.3 Banana Storage:**
- **Designated Banana Zones:** Keep bananas in assigned lockers or fruit baskets.
- **Temperature Control:** Bananas are happiest at room temperature. No freezers!

---

### Section 2: Gadget Safety and Maintenance

**2.1 Gadget Overview:**
- **Types of Gadgets:** From Freeze Rays to Fart Guns, each device requires special care. Be familiar with your assigned gadget (Moo-ti-fi-ka-shun (modification) may occur).

**2.2 Safety Checks:**
- **Pre-Use Inspection:**
  1. Check power levels.
  2. Ensure proper attachment and alignment.
  3. Inspect for any visible damage or banana residue.
  4. Test in designated safe zone before mission.

**2.3 Maintenance Protocol:**
- **Cleaning:** Use soft cloth for cleaning outer surfaces. No water near electrical parts!
- **Repair and Reporting:** Malfunction? Report immediately to Minion Tech Supervisor Kevin.
- **Modification Requests:** For tweaks or moo-di-fi-ka-shuns, submit a request with Gru's approval.

---

### Section 3: Proper "BELLO!" Etiquette

**3.1 Introduction to BELLO!**
- "BELLO!" is the official greeting amongst minions, signifying friendliness and readiness!

**3.2 Mastering the BELLO!:**
- **Tone:** Cheerful, not too loud; always friendly!
- **Timing:** 
  - First meeting of the day - start with BELLO!
  - Group gatherings - use BELLO! to initiate.
- **Practice Drills:** Daily BELLO! drills in the Minion Lounge.

**3.3 Advanced BELLO! Considerations:**
- **Formal Situations:** Add "Boss" after BELLO! when addressing Gru.
- **Mixed Company:** Use BELLO! to differentiate minions from others (no despicable confusion!).

---

**Final Words from Gru:**

"Bello! Minions, you are the heart and bananas of my operations. Follow these guidelines, and we'll do great, great things together! Remember: Keep the teamwork high, safety first, always! Muahaha!"

**Happy minioning!**

---

**End of Manual: Ow-weeeeeeeeeeeee!**

### Minion Worker Result (banana-focused)

**Training Manual for New Minions: Banana-Focused Edition**

### Chapter 1: Welcome to Gru's Evil Operations
**Bello!** Minions! Welcome, welcome! You are now part of Gru's dastardly team of mischief-makers and adventure-seekers! We filled with joy, giggles, and of course, BANANAS! 🍌🍌

### Chapter 2: The Importance of Banana Breaks
Minion no work without banana! Banana break is sacred time. Here’s how you do it:

- **Scheduled Happiness Time**: Take a banana break every two hours. Keeps tummy happy, minion happy!
- **Banana Zone**: In break room, always bananas, never empty.
- **Sharing is Caring**: Share bananas with buddies. Double giggles!

### Chapter 3: Gadget Safety (While Dreaming of Bananas)
Minion have many gadgets. Sometimes dangerous, but no worry if careful!

- **Focus-Shift Protocols**: When banana thought creep in, stop gadget, pause, reset mind on mission.
- **Banana-Proof Gadgets**: Look for gadgets with banana safety stamp: zero mess, zero slips!
- **Buddy System**: Always have banana buddy when testing new gizmos. Two eyes more careful than one!

### Chapter 4: Proper “BELLO!” Etiquette the Banana Way
Saying “BELLO!” even more fun with banana flair!

- **Banana Bello**: Hold banana, wave, say “BELLO!” warm and loud. Receiver more banana-cited! 
- **Banana Dance**: Add wiggle and giggle with greetings. Bring banana, swirl it during wave! 
- **Banana Compliment**: “Belly bello!” is like saying “Your banana is top banana!” High minion praise!

### Chapter 5: Banana Integration for Team Spirit
Banana makes every minion better team player!

- **Banana Challenges**: Organize banana peel sliding contest. Caution: fun, but use safety mats!
- **Banana Jam Session**: Play music with bananas, turn into microphone 🎤. Sing “B-A-N-A-N-A, yes we can-a!”
- **Team Building with Banana**: Create banana art to decorate lair. Frame banana portraits, very sophisticated!

### Chapter 6: Troubleshooting Common Banana Issues
Things happen! But smart minion ready!

- **Banana Shortage Panic?**: Check secret stash behind lab equipment – always emergency bananas!
- **Squished Banana**: No sadness! Make banana smoothie or use as perfect goo in traps for enemies!
- **Banana Stealing?**: Use CCTV – “Comprende, Tom?” (Always Tom... 🙄)

### Chapter 7: Final Words
Never forget: Banana in heart, minion in heart. With bananas, we conquer world but first, we giggle! We welcome you, new friends, with open arms and loaded banana bags. Together, we'll make mischief and maybe, sometimes, good too. Happy adventures!

### Conclusion:
BANANA! Congratulations, minion! You ready for amazing journey with Gru. Remember, have fun, stay safe, and eat all the bananas! Let's go minion, to adventure! 🎉🍌

**End Manual**